In [ ]:
%%javascript
$('<style>#appmode-leave{display:none}</style>').appendTo('head');

In [ ]:
from nanohub.uidl.ipywidgets import buildJSX
import ipywidgets as w
import nanohub.remote as nr
import os

def FileLink( filepath, path="files"):
   global jupyter_notebook_url
   import os
   if os.path.exists(filepath):
       bp = os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID'])
       ap = os.path.abspath( filepath )
       if ap.startswith(bp):
           link =  "/".join(jupyter_notebook_url.split("/",8)[:7])
           link += "/" + path + "/" + os.path.relpath(ap,bp)
           return link
       else:
           raise Exception("Dont have access to the file")
   else:
       raise Exception(filepath + " does not exists")
        


In [ ]:
Divider = buildJSX('''
<Material.AppBar position="relative" style="{style:object({})}">
<Material.Toolbar variant="dense">
<img src="{image:string( )}"></img>
<Material.Typography variant="subtitle1" noWrap="false" component="div">
<div style="text-transform:uppercase">{title:string(asd)}</div>
</Material.Typography>
</Material.Toolbar>
</Material.AppBar>
''', debugger=False);


In [ ]:
DashboardCard = buildJSX('''
<Material.Card>
  <Material.CardMedia
    image="{image:string(default.jpg)}"
    title="{image_title:string(default)}"
    style="{image_style:object({})}"
  >
  </Material.CardMedia>
  <Material.CardContent>
    <Material.Typography color="textSecondary">
      {title:string(default)}
    </Material.Typography>
    <Material.Typography component="p" variant="body2">
      {description:string( )}
    </Material.Typography>
  </Material.CardContent>
  <Material.CardActions>
    <a target="blank" href="{url:string('/')}">{url_description:string( )}</a>
  </Material.CardActions>
</Material.Card>
''', debugger=False);


In [ ]:
UserCard = buildJSX('''
<Material.Card>
  <Material.CardMedia
    image="{image:string(default.jpg)}"
    style="{image_style:object({})}"
  >
  </Material.CardMedia>
  <Material.CardContent>
    <Material.Typography color="textSecondary">
      {name:string(default)}
    </Material.Typography>
    <Material.Typography component="p" variant="body2">
      {organization:string( )}
    </Material.Typography>
    <Material.Typography component="p" variant="body2">
      {email:string( )}
    </Material.Typography>
  </Material.CardContent>
</Material.Card>
''', debugger=False);


In [ ]:
ProjectCard = buildJSX('''
<Material.Card variant="outlined" style="display:flex;width:320px">
  <Material.CardContent>
    <Material.Typography color="textSecondary">
      <a target="blank" href="{url:string('/')}">{title:string(default)}</a>
    </Material.Typography>
    <Material.Typography component="p" variant="body2">
      {owner:string( )}
    </Material.Typography>
    <Material.Typography component="p" variant="body2">
      {description:string( )}
    </Material.Typography>
  </Material.CardContent>
  <div
    style="{image_style:object({})}"
  >
  </div>
</Material.Card>
''', debugger=False);


In [ ]:
auth_data = {'grant_type' : 'tool'}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
session = nr.Session(auth_data, url='https://cdmhub.org/api/')
req_json = session.requestGet('/projects/list?verbose=1')
req_json = req_json.json()
projects = {}
for p in req_json['projects']:
    #if 'a3dconsortium' in p['alias']:
    if 'cams' in p['alias']:
        path = '/data/projects/' + p['alias'] + "/files"
        if os.path.exists(path):
            p['path'] = path
            projects[p['alias']] = p
req_json = session.requestGet('/members/' + str(os.getuid()))
data = req_json.json()
data = data['profile']

In [ ]:
results = DashboardCard(
    image="https://images.unsplash.com/photo-1574656365304-8fc1be04a3bd?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8OHx8Y29tcG9zaXRlfGVufDB8fDB8fA%3D%3D&auto=format&fit=crop&w=500&q=60",
    image_style = {"height":"200px"},
    title = "VIEW RESULTS",
    description = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam molestie arcu ligula. Curabitur nec enim ullamcorper, mollis sem eu, luctus nibh. Cras porttitor dolor nunc, eu egestas sem pharetra non. Integer sed posuere neque. Integer posuere interdum ipsum, sit amet fermentum nulla dictum eu. Vestibulum mollis at turpis nec auctor.",
    url = FileLink('a3dWizard.ipynb', "apps"),
    url_description = "OPEN VISUALIZATION MODULE ..",
    layout = w.Layout(width="100%")
)


In [ ]:
jobs = DashboardCard(
    image="https://images.unsplash.com/photo-1611567434593-6552b11d1b02?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8MXx8cG9seW1lcnxlbnwwfHwwfHw%3D&auto=format&fit=crop&w=500&q=60",
    image_style = {"height":"200px"},
    title = "NEW JOBS",
    description = "Nunc at scelerisque tortor. Cras suscipit non lectus at posuere. Vivamus vitae odio eu leo condimentum dictum eget a felis. Phasellus posuere viverra tincidunt. Sed luctus tortor ac gravida mollis",
    url = FileLink('SubmitJobs.ipynb', "apps"),
    url_description = "OPEN NEW JOB MODULE ..",
    layout = w.Layout(width="100%")
)

In [ ]:
user = UserCard(
    image=data['picture']['thumb'].replace("https://cdmhub.org/",""),
    image_style = {"width":"200px", "height":"200px"},
    name = data['name'],
    organization = data["organization"],
    email = data["email"]
)

In [ ]:
children = []
colors = ['#912b5e','#915e2b','#2b915e','#2b5e91']
c=0
for i, p in projects.items():
    children.append(ProjectCard(
        image_style = {"width":"100px", "backgroundColor":colors[c % 4]},
        title = projects[i]['title'],
        owner = projects[i]['title'],
        description = projects[i]['description'],
        url = projects[i]['uri'],
    ))
    c+=1

In [ ]:

box_layout = w.Layout(overflow='scroll hidden',
                    width='100%',
                    height='',
                    display='flex')
w.VBox([
    Divider(title = "", style={'background-color': '#2B5E91'}, image=FileLink('./logoA3D.png')),
    w.HBox([
        w.VBox([user], layout = w.Layout(flex="1 0 auto")),
        w.VBox([
            Divider(title = "projects", style={'background-color': '#2B5E91'}, image='data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs='),
            w.HBox(children),
            Divider(title = "Modules", style={'background-color': '#2B5E91'}, image = 'data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs='),
            w.HBox([jobs,results], layout=box_layout)
        ])
    ])
    
])